In [1]:
import pandas as pd
import pandas.io.sql as psql
import os
import sys
from datetime import date
current_date = date.today()
#%load_ext autotime
import re
import csv
import pandas as pd
import numpy as np
import os
from glob import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
#from matplotlib_venn import venn2, venn3
import warnings
import time
import datetime as dt
import itertools
from more_itertools import unique_everseen
warnings.filterwarnings("ignore")
#import pandasql as ps
import seaborn as sns
import json
from ast import literal_eval
import ast
from fuzzywuzzy import process
from nltk.corpus import words

In [2]:
total = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\total_summary_stats_updated.csv")

In [3]:
patient_id = total["patient_id"].to_list()
patient_id = patient_id[:10]

In [4]:
matches = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\1\apoe_1_matches.csv")

In [ ]:
# ONLY RUN ONCE #

# vertifying that all match records can be found
for i in range(len(patient_id)):
    specific_matches = matches[matches["PatientID"] == patient_id[i]]
    if (len(specific_matches) <= 0):
        print("NOT FOUND: ", patient_id[i])
    #specific_matches.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\1\Specific_Matches\patient_{}.csv".format(i+1))

In [5]:
p1 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\1\Specific_Matches\patient_1.csv")

In [6]:
unique_ids = matches["PatientID"].unique().tolist()
print(len(unique_ids))

2890


In [7]:
def get_match(query, choices, limit = 1):
    results = process.extract(query, choices, limit = limit)
    return results

In [9]:
# interesting match cases
# patient_2 in specific study row 6, cream that has mci inside it

In [97]:
subset_loc = p1[p1["PatientID"] == 'Z6352398']['regex_location'].to_list()
#print(subset_loc)
encounters = p1[p1["PatientID"] == 'Z6352398']['PatientEncounterID'].unique().tolist()

#print(len(subset_loc))
#print(subset_loc)
#print(subset_loc[0])
for i in range(len(subset_loc)):
    subset_loc[i] = subset_loc[i][1:len(subset_loc[i]) - 1]
    subset_loc[i] = eval(subset_loc[i])
    #subset_loc[i] = tuple(map(int, subset_loc[i].split(", ")))
    
subset_loc = pd.Series(subset_loc)
subset_loc = subset_loc.unique().tolist()

subset_loc_clone = subset_loc

print(subset_loc)
print("\n", subset_loc[0][0])
print(len(subset_loc))
if (len(subset_loc) != len(encounters)):
    print("Houston, we got a problem", i, len(subset_loc), len(encounters))

total_merged = []  

for i in range(len(subset_loc)):
    merged = []

    start = 0
    end = 0
    length = 0
    
    if (len(subset_loc[i]) == 2):
        start = subset_loc[i][0]
        end = subset_loc[i][1]
        length = len(p1[(p1["regex_location"] == "[" +str(subset_loc[i]) + "]") & (p1["PatientEncounterID"] == int(encounters[i]))]['NoteTXT'].values[0])
        if (start < 100):
            start = 100
        elif (end > length - 100):
             end = length - 100
        subset_loc[i] = [(start - 100, end + 100)]
        
    else:
        loc_list = []
        for j in range(len(subset_loc[i])):
            start = subset_loc[i][j][0]
            end = subset_loc[i][j][1]
            loc_list.append((start, end))
        length = len(p1[(p1["regex_location"] == str(loc_list)) & (p1["PatientEncounterID"] == int(encounters[i]))]['NoteTXT'].values[1])
        for k in range(len(loc_list)):
            start = loc_list[k][0]
            end = loc_list[k][1]
            if (start < 100):
                start = 100
            elif (end > length - 100):
                end = length - 100
            loc_list[k] = (start - 100, end + 100)
        subset_loc[i] = loc_list
    
    subset_loc[i].sort(key = lambda x : x[0])
    
    print("\nSUBSET_LOC", subset_loc[i])
    
    for interval in subset_loc[i]:
        if not merged:
            merged.append(interval)
            continue

        prevInterval = merged.pop()
        if prevInterval[0] <= interval[0] <= prevInterval[1]:
            startTime = prevInterval[0]
            endTime = prevInterval[1] if prevInterval[1] > interval[1] else interval[1]
            merged.append((startTime, endTime))
        else:
            merged.append(prevInterval)
            merged.append(interval)
    total_merged.append(merged)
    
total_merged
#     print("start", start)
#     print("end", end)
    #print("matches, ", p1[(p1["regex_location"] == "[" +str(subset_loc[j]) + "]") & (p1["PatientEncounterID"] == int(encounters[j]))]['NoteTXT'].values)
    #print("length, ", length) 
    

[(5850, 5858), (3643, 3649), (6049, 6057), (3810, 3816), (5554, 5562), (5634, 5642), (5492, 5500), (5891, 5899), (5790, 5798), (6170, 6178), (6128, 6136), (6061, 6069), (6355, 6363), (3947, 3950), (8741, 8749), ((1048, 1052), (4878, 4887), (5104, 5110), (5114, 5123), (5227, 5231)), ((5467, 5476), (5693, 5699), (5703, 5712)), (1984, 1990), (859, 862), ((281, 290), (579, 588), (2148, 2154), (2163, 2169), (6284, 6290), (6684, 6693), (6718, 6721), (7550, 7559), (7816, 7820), (7828, 7832), (9964, 9972), (11292, 11301), (11383, 11403), (11514, 11523), (11544, 11548), (12218, 12238), (12383, 12392), (12487, 12507), (12509, 12512), (12519, 12527)), (313, 319), ((7509, 7518), (7735, 7741), (7745, 7754), (7858, 7862)), (1473, 1476)]

 5850
23

SUBSET_LOC [(5750, 5958)]

SUBSET_LOC [(3543, 3749)]

SUBSET_LOC [(5949, 6157)]

SUBSET_LOC [(3710, 3916)]

SUBSET_LOC [(5454, 5662)]

SUBSET_LOC [(5534, 5742)]

SUBSET_LOC [(5392, 5600)]

SUBSET_LOC [(5791, 5999)]

SUBSET_LOC [(5690, 5898)]

SUBSET_LOC [(

[[(5750, 5958)],
 [(3543, 3749)],
 [(5949, 6157)],
 [(3710, 3916)],
 [(5454, 5662)],
 [(5534, 5742)],
 [(5392, 5600)],
 [(5791, 5999)],
 [(5690, 5898)],
 [(6070, 6278)],
 [(6028, 6236)],
 [(5961, 6169)],
 [(6255, 6463)],
 [(3847, 4050)],
 [(8641, 8849)],
 [(948, 1152), (4778, 4987), (5004, 5331)],
 [(5367, 5576), (5593, 5812)],
 [(1884, 2090)],
 [(759, 962)],
 [(181, 390),
  (479, 688),
  (2048, 2269),
  (6184, 6390),
  (6584, 6821),
  (7450, 7659),
  (7716, 7932),
  (9864, 10072),
  (11192, 11648),
  (12118, 12627)],
 [(213, 419)],
 [(7409, 7618), (7635, 7962)],
 [(1373, 1576)]]

In [81]:
x = ((7509, 7518), (7735, 7741), (7745, 7754), (7858, 7862))
y = []
for i in range(len(x)):
    a = x[i][0]
    b = x[i][1]
    y.append((a,b))
print(str(y))
z = len(p1[(p1["regex_location"] == str(y)) & (p1["PatientEncounterID"] == int(encounters[21]))]['NoteTXT'].values[1])
z

[(7509, 7518), (7735, 7741), (7745, 7754), (7858, 7862)]


9187

In [99]:
x = [(181, 390),
  (479, 688),
  (2048, 2269),
  (6184, 6390),
  (6584, 6821),
  (7450, 7659),
  (7716, 7932),
  (9864, 10072),
  (11192, 11648),
  (12118, 12627)]
y = []
for i in range(len(x)):
    a = x[i][0]
    b = x[i][1]
    y.append((a,b))
print(y)

[(181, 390), (479, 688), (2048, 2269), (6184, 6390), (6584, 6821), (7450, 7659), (7716, 7932), (9864, 10072), (11192, 11648), (12118, 12627)]


In [40]:
len(p1[(p1["regex_location"] == "[" +str(subset_loc[1]) + "]") & (p1["PatientEncounterID"] == int(encounters[1]))]['NoteTXT'].values[0])

5850